In [1]:
import pandas as pd
import numpy as np
import sys
%matplotlib inline


In [2]:
print("Pandas Version" +pd.__version__)
print("Python Version"+sys.version)

Pandas Version0.21.0
Python Version3.6.3 |Anaconda custom (64-bit)| (default, Nov  8 2017, 18:10:31) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


# Compute
Getting the sum and length of a group

In [3]:
df=pd.DataFrame({"group1":["a","a","b",'b']
                , 'value':[10,20,30,40]})
df

,group1,value
0,a,10
1,a,20
2,b,30
3,b,40


In [4]:
group=df.groupby("group1")
group.agg([len,sum])

value    
         len sum
group1          
a          2  30
b          2  70

In [5]:
group.agg([len])

,value
,len
group1,
a,2
b,2


In [6]:
group.agg([sum])

,value
,sum
group1,
a,30
b,70


## Add a column that's equal to the sum of the group

In [9]:
df=pd.DataFrame({"labels":["a","a","b",'b']
                , 'value':[10,20,30,40]})
df

,labels,value
0,a,10
1,a,20
2,b,30
3,b,40


In [10]:
group=df.groupby("labels")['value']
group

In [15]:
df['value.sum']=group.transform('sum')
df['value.max']=group.transform("max")
df['value.min']=group.transform("min")
df['value.mean']=group.transform("mean")
df

,labels,value,value.sum,value.max,value.min,value.mean
0,a,10,30,20,10,15
1,a,20,30,20,10,15
2,b,30,70,40,30,35
3,b,40,70,40,30,35


## Getting the month name out of the date column

In [20]:
df=pd.DataFrame({
    'col1':[pd.Timestamp('20130102000030'),
           pd.Timestamp('2013-02-03 00:00:30'),
           pd.Timestamp('3/4/2013 000030')]
})
df.columns=['Time']
df

,Time
0,2013-01-02 00:00:30
1,2013-02-03 00:00:30
2,2013-03-04 00:00:30


In [21]:
df['MonthNumber']=df['Time'].apply(lambda x: x.month)
df['Day']=df['Time'].apply(lambda x:x.day)
df['Year']=df['Time'].apply(lambda x:x.year)
df["MonthName"]=df['Time'].apply(lambda x:x.strftime('%B'))
df["WeekDay"]=df["Time"].apply(lambda x:x.strftime("%A"))
df

,Time,MonthNumber,Day,Year,MonthName,WeekDay
0,2013-01-02 00:00:30,1,2,2013,January,Wednesday
1,2013-02-03 00:00:30,2,3,2013,February,Sunday
2,2013-03-04 00:00:30,3,4,2013,March,Monday


# Creating column based on two other columns


In [22]:
df = pd.DataFrame({'col1':['minus','minus','positive','nan'],
                'col2':[10,20,30,40]
                })
df


,col1,col2
0,minus,10
1,minus,20
2,positive,30
3,nan,40


In [24]:
df['col3']=df['col2']*df['col1'].apply(lambda x: -1 if x=="minus" else (1 if x=="positive" else np.nan))
df


,col1,col2,col3
0,minus,10,-10.0
1,minus,20,-20.0
2,positive,30,30.0
3,nan,40,NaN


## Applying a function to a group and add the result to the original data frame

In [25]:
df = pd.DataFrame({'group1':['a','a','a','b','b','b'],
                       'group2':['c','c','d','d','d','e'],
                       'value1':[1.1,2,3,4,5,6],
                       'value2':[7.1,8,9,10,11,12]
})

df

,group1,group2,value1,value2
0,a,c,1.1,7.1
1,a,c,2.0,8.0
2,a,d,3.0,9.0
3,b,d,4.0,10.0
4,b,d,5.0,11.0
5,b,e,6.0,12.0


In [38]:
group=df.groupby(['group1','group2'])

def Half(x):
    return x.sum()

df['new']=group['value1'].transform(Half)
df

df['new1']=group['value2'].transform(Half)
df

def sumhalf(x):
    return (x/2)


df['new2']=group['new1'].transform(sumhalf)
df

,group1,group2,value1,value2,new,new1,new2
0,a,c,1.1,7.1,3.1,15.1,7.55
1,a,c,2.0,8.0,3.1,15.1,7.55
2,a,d,3.0,9.0,3.0,9.0,4.50
3,b,d,4.0,10.0,9.0,21.0,10.50
4,b,d,5.0,11.0,9.0,21.0,10.50
5,b,e,6.0,12.0,6.0,12.0,6.00


In [42]:
#passing multiple functions
def HalfPlus(x):
    return x.sum()+1

newcol=group['value1'].agg([Half,HalfPlus])
newcol

Half  HalfPlus
group1 group2                
a      c        3.1       4.1
       d        3.0       4.0
b      d        9.0      10.0
       e        6.0       7.0

In [43]:
#mergine the results so far
df.merge(newcol,left_on=['group1','group2'],right_index=True)

,group1,group2,value1,value2,new,new1,new2,Half,HalfPlus
0,a,c,1.1,7.1,3.1,15.1,7.55,3.1,4.1
1,a,c,2.0,8.0,3.1,15.1,7.55,3.1,4.1
2,a,d,3.0,9.0,3.0,9.0,4.50,3.0,4.0
3,b,d,4.0,10.0,9.0,21.0,10.50,9.0,10.0
4,b,d,5.0,11.0,9.0,21.0,10.50,9.0,10.0
5,b,e,6.0,12.0,6.0,12.0,6.00,6.0,7.0


## Adding two data frames and not get null values?

In [44]:
df1=pd.DataFrame(data=[26371,1755,2],index=[-9999,240,138.99],columns=['value'])

df1

,value
-9999.00,26371
240.00,1755
138.99,2


In [45]:
df2 = pd.DataFrame(data=[26371, 1755, 6, 4], index=[-9999, 240, 113.03, 110], columns=['value'])
df2

,value
-9999.00,26371
240.00,1755
113.03,6
110.00,4


In [46]:
df1+df2

,value
-9999.00,52742.0
110.00,NaN
113.03,NaN
138.99,NaN
240.00,3510.0


In [47]:
df1.add(df2,fill_value=0)

,value
-9999.00,52742.0
110.00,4.0
113.03,6.0
138.99,2.0
240.00,3510.0
